# Run cellpose on a single file

In [ ]:
czi_file_path = '/home/ubuntu/Projects/data/uploads/grzegorz.kossakowski@gmail.com/FISH1_BDNF488_1_cLTP_3_CA-gkk FISH1_BDNF488_1_cLTP_2_CA-gkk/FISH1_BDNF488_1_cLTP_2_CA-gkk.czi'
out_mask_filepath = '/home/ubuntu/tmp/masks_3d_FISH1_BDNF488_1_cLTP_3_CA.npy'
# we assume we make use of only one channel (DAPI) from the input
nuclei_channel = 1
mask_volume_threshold_of_max = 0.05
diameter = 110

In [ ]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

from urllib.parse import urlparse
import torch
from cellpose import utils

from pathlib import Path

# use_GPU = utils.use_gpu()
# print('GPU activated? %d'%use_GPU)


In [ ]:
assert Path(czi_file_path).suffix == '.czi', czi_file_path
assert Path(czi_file_path).exists(), czi_file_path

In [ ]:
base_filename = Path(czi_file_path).stem
base_filename

## save output to *_seg.npy

you will see the files save in the Files tab and you can download them from there

In [ ]:
# TODO(gkk): we might want to save this 
# from cellpose import io

# io.masks_flows_to_seg(imgs, masks, flows, diams, files, channels)

# Read the input data

In [ ]:
import tifffile
tifffile.__version__

In [ ]:
from tifffile import imread

In [ ]:
from aicsimageio import AICSImage

In [ ]:
# swap axes so we have the (Z, channels, Y, X) shape
# TODO: make this automatic
data = np.swapaxes(AICSImage(czi_file_path).data.squeeze(), 0, 1)
data.shape

## run cellpose 3D mode on CZI image

In [ ]:
z_start_idx, z_end_idx = 0,data.shape[0]
z_start_idx, z_end_idx

In [ ]:
imgs = data[z_start_idx:z_end_idx,nuclei_channel,:,:]

In [ ]:
imgs.shape

In [ ]:
channels = len(imgs)*[[0,0]]

In [ ]:

%%time
# test 3D stack
from cellpose import models

model = models.Cellpose(gpu=True, model_type='cyto')

# in this example I'm using a random matrix, put your own data here
# data = np.random.randn(120,512,512).astype(np.float32)
# data = imread('/home/gkk/ada_lsm_test_squeeze.tif')

# with 3D you have to set the diameter manually (no auto detect)
#imgs, diameter=110, #flow_threshddold=, 
#                                         cellprob_threshold=0.5, resample=True,
#                                          flow_threshold=0.2,
#                                         stitch_threshold=0.4,
#                                         channels=channels
masks, flows, styles, diams = model.eval(imgs, channels=channels,
                                         diameter=110, 
                                         do_3D=True, 
                                         resample=True,
                                         cellprob_threshold=0.5,
                                         flow_threshold=0.5,
#                                          anisotropy=3.0,
#                                         boundary_threshold=0.5
#                                          min_size=80.0,
                                         batch_size=2)

In [ ]:
masks.shape

In [ ]:
np.unique(masks)

In [ ]:
from cellpose import plot
def display_results(imgs, masks, flows):
    nimg = len(imgs)
    for idx in range(nimg):
        maski = masks[idx]
        flowi = flows[idx]

        fig = plt.figure(figsize=(12,5))
        plot.show_segmentation(fig, imgs[idx], maski, flowi, channels=channels[idx])
        plt.tight_layout()
        plt.show()

In [ ]:
display_results(imgs, masks, flows[0])

## Remap mask ids
Remap mask ids to "compact" integers (from 0..len(unique_ids) range)

In [ ]:
def compact_integer_values(a):
    uniqs = np.unique(a)
    max_value = uniqs.max()
    assert max_value < 100000, max_value # some reasonable threshold that assures we do not allocate unreasonable amounts of memory below
    compact_for_uniqs = np.zeros(uniqs.max()+1, dtype=np.int)
    compact_for_uniqs[uniqs] = range(len(uniqs)) # each uniq receives a new index from the range
    a_compact_values = compact_for_uniqs[a]
    return a_compact_values

In [ ]:
# masks = compact_integer_values(masks)

## Mask volume estimate through voxel counting

In [ ]:
masks_orig = masks
masks = np.copy(masks_orig)

In [ ]:
mask_indices, mask_volume = np.unique(masks,return_counts=True)
mask_indices, mask_volume = mask_indices[1:], mask_volume[1:] # zero is background so we drop it for volume analysis
mask_indices, mask_volume

In [ ]:
plt.bar(range(len(mask_volume)), mask_volume)
plt.title('Volume distribution')
plt.show()

In [ ]:
mask_volume_threshold = mask_volume.max()*mask_volume_threshold_of_max
mask_volume_threshold

In [ ]:
np.sum(mask_volume > mask_volume_threshold)

In [ ]:
masks_indices_above_threshold = mask_indices[mask_volume > mask_volume_threshold]

In [ ]:
masks[np.isin(masks, masks_indices_above_threshold, invert=True)] = 0

In [ ]:
np.unique(masks)

In [ ]:
compact_integer_values(masks)

In [ ]:
masks = masks_orig

Put the exploratory code from the above into a reusable function

In [ ]:
from IPython.display import display
def zero_small_volume_masks(masks, mask_volume_threshold_of_max):
    mask_indices, mask_volume = np.unique(masks,return_counts=True)
    mask_indices, mask_volume = mask_indices[1:], mask_volume[1:] # zero is background so we drop it for volume analysis
    display(mask_indices, mask_volume)
    
    plt.bar(range(len(mask_volume)), mask_volume)
    plt.title('Volume distribution')
    plt.show()
    
    mask_volume_threshold = mask_volume.max()*mask_volume_threshold_of_max
    
    import pandas as pd
    df = pd.DataFrame(mask_volume, columns=['volume'])
    display(df)
    
    df['% of max'] = (df['volume'] / df['volume'].max() * 100).round(1)
    display(df)
    
    masks_indices_above_threshold = mask_indices[mask_volume > mask_volume_threshold]
    masks_large_only = np.copy(masks)
    masks_large_only[np.isin(masks_large_only, masks_indices_above_threshold, invert=True)] = 0
    
    masks_large_only = compact_integer_values(masks_large_only)
    
    display(np.unique(masks_large_only))
    
    return masks_large_only

### Remove small volume outliers

In [ ]:
masks = zero_small_volume_masks(masks, mask_volume_threshold_of_max)

In [ ]:
np.unique(masks)

### Save final masks: compacted and without small volume outliers

In [ ]:
np.save(out_mask_filepath, masks.astype(np.int8))